In [ ]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd

# Configurações iniciais
ticket = 'KEPL3'
msft = yf.Ticker(ticket + '.SA')

# Dados da ação
data = msft.info
current_price = data.get('currentPrice', None)
target_high = data.get('targetHighPrice', None)
target_low = data.get('targetLowPrice', None)
target_mean = data.get('targetMeanPrice', None)

# Obter recomendações dos analistas
recommendations = msft.recommendations

if recommendations is not None and not recommendations.empty:
    # Contando as recomendações mais recentes
    recent_recommendations = recommendations.tail(50)  
    
    buy_count = recent_recommendations['buy'][0]
    hold_count = recent_recommendations['sell'][0]
    sell_count = recent_recommendations['hold'][0]
    stg_buy = recent_recommendations['strongBuy'][0]
else:
    buy_count = hold_count = sell_count = str_buy_count = 0

# Verificação de dados disponíveis
if current_price and target_high and target_low and target_mean:
    # Calculando as diferenças percentuais
    differences = {
        'Target Low': (target_low - current_price) / current_price * 100,
        'Target Mean': (target_mean - current_price) / current_price * 100,
        'Target High': (target_high - current_price) / current_price * 100,
    }

    # Organizando os dados em um DataFrame para o gráfico
    price_comparison = pd.DataFrame({
        'Type': ['Current Price', 'Target Low', 'Target Mean', 'Target High'],
        'Price': [current_price, target_low, target_mean, target_high],
        'Difference (%)': [0] + [round(differences[key], 2) for key in differences.keys()]
    })

    # Criando o gráfico Plotly
    fig = go.Figure()

    # Adicionando os dados ao gráfico
    for i, row in price_comparison.iterrows():
        color = 'blue' if row['Type'] == 'Current Price' else ('green' if row['Difference (%)'] > 0 else 'red')
        fig.add_trace(go.Bar(
            x=[row['Type']],
            y=[row['Price']],
            marker_color=color,
            text=f"{row['Difference (%)']}%",
            textposition='outside',
            name=row['Type']
        ))

    # Adicionando recomendações dos analistas como texto adicional no gráfico
    recommendations_text = (
        f"Analyst Recommendations:\n"
        f"Buy: {buy_count}\n"
        f"Hold: {hold_count}\n"
        f"Sell: {sell_count}\n"
        f"Strong Sell: {stg_buy}"
    )

    fig.add_annotation(
        x=1.5,  # Posição horizontal (ajuste conforme necessário)
        y=max(price_comparison['Price']) * 1.1,  # Posição vertical acima das barras
        text=recommendations_text,
        showarrow=False,
        font=dict(size=12, color="black"),
        align="left",
        bgcolor="lightyellow",
        bordercolor="black",
        borderwidth=1
    )

    # Configurações do layout
    fig.update_layout(
        title=f'Price Comparison and Analyst Recommendations for {ticket}',
        xaxis_title='Type',
        yaxis_title='Price (BRL)',
        template='plotly_white',
        font=dict(size=14),
        showlegend=False,
        barmode='group'
    )

    # Exibindo o gráfico
    fig.show()
else:
    print("Dados insuficientes para gerar o gráfico. Verifique se os preços-alvo e o preço atual estão disponíveis.")


In [43]:
recent_recommendations

,period,strongBuy,buy,hold,sell,strongSell
0,0m,0,2,0,0,0


In [16]:
recommendations

,period,strongBuy,buy,hold,sell,strongSell
0,0m,0,2,0,0,0


In [ ]:
#result : year: 2025, stock: KEPL3.SA, Low: 26%, Mean: 47%, High: 68.07%